In [3]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [5]:
loader = TextLoader("data/technology.txt")
data = loader.load()
data

[Document(metadata={'source': 'data/technology.txt'}, page_content='Technology is a transformative force, making life easier, faster, and more connected through innovations like smartphones, the internet, and AI, revolutionizing fields from medicine to education, but it also presents challenges such as privacy risks, job displacement, and screen addiction, demanding responsible use to balance benefits with potential harms for a better future. \nThe Double-Edged Sword of Modern Technology\nTechnology, from ancient tools to today\'s AI, has always shaped human civilization, but its rapid evolution in the digital age presents both unprecedented opportunities and significant challenges. It\'s a powerful catalyst for progress, improving efficiency, communication, and access to knowledge, yet it also necessitates careful consideration of its societal and personal impacts, making responsible usage paramount. \nBenefits: Convenience, Connection, and Progress\nEnhanced Convenience & Efficiency:

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)
splits

[Document(metadata={'source': 'data/technology.txt'}, page_content='Technology is a transformative force, making life easier, faster, and more connected through innovations like smartphones, the internet, and AI, revolutionizing fields from medicine to education, but it also presents challenges such as privacy risks, job displacement, and screen addiction, demanding responsible use to balance benefits with potential harms for a better future. \nThe Double-Edged Sword of Modern Technology'),
 Document(metadata={'source': 'data/technology.txt'}, page_content="Technology, from ancient tools to today's AI, has always shaped human civilization, but its rapid evolution in the digital age presents both unprecedented opportunities and significant challenges. It's a powerful catalyst for progress, improving efficiency, communication, and access to knowledge, yet it also necessitates careful consideration of its societal and personal impacts, making responsible usage paramount. \nBenefits: Conve

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-O11f3XoTMixcX1DWAwy6fcBZf5lqdf3cxphI337AZ4NJLiNHPlrRamuinfIQVUWvIh_ed5LzyfpKT3BlbkFJNcHarqmCgmaQ6L0Feb1aftJwIisQ1MlmNZaXSBthLLoZSTBwspxBLRTHWoQdUPh1KXjyQW5BEA"


In [12]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",dimensions=1024)
vector_db = Chroma.from_documents(splits, embedding=embeddings, collection_name="tech_docs")
vector_db

/home/maniram/workspace/python/langchain-projects/.venv/lib/python3.12/site-packages/langchain_community/embeddings/openai.py:271: UserWarning: WARNING! dimensions is not default parameter.
                    dimensions was transferred to model_kwargs.
                    Please confirm that dimensions is what you intended.
  warnings.warn(


In [13]:
query = "what is the benefit os techonology?"
docs = vector_db.similarity_search(query)
print(len(docs))
docs[0].page_content


4


"Technology, from ancient tools to today's AI, has always shaped human civilization, but its rapid evolution in the digital age presents both unprecedented opportunities and significant challenges. It's a powerful catalyst for progress, improving efficiency, communication, and access to knowledge, yet it also necessitates careful consideration of its societal and personal impacts, making responsible usage paramount. \nBenefits: Convenience, Connection, and Progress"

In [14]:
## saving to the disk
vectordb = Chroma.from_documents(documents=splits, embedding=embeddings, 
collection_name="tech_docs", persist_directory="./chroma_db")

In [21]:
db_new = Chroma(persist_directory="./chroma_db", embedding_function=embeddings, collection_name="tech_docs")
docs = db_new.similarity_search(query)
print(len(docs))
print(docs[0].page_content)
docs

4
Technology, from ancient tools to today's AI, has always shaped human civilization, but its rapid evolution in the digital age presents both unprecedented opportunities and significant challenges. It's a powerful catalyst for progress, improving efficiency, communication, and access to knowledge, yet it also necessitates careful consideration of its societal and personal impacts, making responsible usage paramount. 
Benefits: Convenience, Connection, and Progress


[Document(id='25c22bf1-fd6d-4645-a22d-6531d8693f4b', metadata={'source': 'data/technology.txt'}, page_content="Technology, from ancient tools to today's AI, has always shaped human civilization, but its rapid evolution in the digital age presents both unprecedented opportunities and significant challenges. It's a powerful catalyst for progress, improving efficiency, communication, and access to knowledge, yet it also necessitates careful consideration of its societal and personal impacts, making responsible usage paramount. \nBenefits: Convenience, Connection, and Progress"),
 Document(id='b549ae42-ebfa-4846-995a-e9bdb429a389', metadata={'source': 'data/technology.txt'}, page_content='Technology is an indispensable tool that has profoundly improved human life, offering comfort, knowledge, and progress. However, its double-edged nature demands mindful engagement; balancing screen time with real-world activities and understanding its ethical implications are crucial. By using technology 

In [24]:
docs_with_scores = db_new.similarity_search_with_score(query)
print(len(docs_with_scores))
docs_with_scores

4


[(Document(id='25c22bf1-fd6d-4645-a22d-6531d8693f4b', metadata={'source': 'data/technology.txt'}, page_content="Technology, from ancient tools to today's AI, has always shaped human civilization, but its rapid evolution in the digital age presents both unprecedented opportunities and significant challenges. It's a powerful catalyst for progress, improving efficiency, communication, and access to knowledge, yet it also necessitates careful consideration of its societal and personal impacts, making responsible usage paramount. \nBenefits: Convenience, Connection, and Progress"),
  0.823413610458374),
 (Document(id='b549ae42-ebfa-4846-995a-e9bdb429a389', metadata={'source': 'data/technology.txt'}, page_content='Technology is an indispensable tool that has profoundly improved human life, offering comfort, knowledge, and progress. However, its double-edged nature demands mindful engagement; balancing screen time with real-world activities and understanding its ethical implications are cruci

In [25]:
## Retriver option
retriever = vectordb.as_retriever()
retriever.invoke(query)[0].page_content

"Technology, from ancient tools to today's AI, has always shaped human civilization, but its rapid evolution in the digital age presents both unprecedented opportunities and significant challenges. It's a powerful catalyst for progress, improving efficiency, communication, and access to knowledge, yet it also necessitates careful consideration of its societal and personal impacts, making responsible usage paramount. \nBenefits: Convenience, Connection, and Progress"